# 🔧 Solución de Errores de Deploy en Render

Este notebook documenta la solución para el error `Cannot find module 'glob'` en el deploy de Render.

## 📋 Problema Detectado
```
Error: Cannot find module 'glob'
Require stack:
- /opt/render/project/src/src/generate.js
```

## 1. Verificar Dependencias del Proyecto

Primero verificamos qué dependencias están listadas en el package.json y cuáles están instaladas.

In [ ]:
# Verificar dependencias en package.json
echo "📦 Dependencias listadas en package.json:"
cat package.json | grep -A 20 '"dependencies"'

echo "\n📂 Verificar si node_modules existe:"
ls -la | grep node_modules

echo "\n🔍 Buscar módulo 'glob' específicamente:"
ls node_modules/ | grep glob || echo "❌ Módulo 'glob' no encontrado"

## 2. Instalar Módulos Faltantes

El módulo 'glob' está listado en las dependencias pero no se está instalando correctamente.

In [ ]:
# Instalar el módulo glob específicamente
echo "🔧 Instalando módulo 'glob'..."
npm install glob

echo "\n✅ Verificar instalación:"
npm list glob

## 3. Revisar package.json

Analizamos el archivo package.json para verificar que todas las dependencias estén correctamente declaradas.

In [ ]:
// Verificar que todas las dependencias requeridas estén en package.json
const fs = require('fs');
const packageJson = JSON.parse(fs.readFileSync('package.json', 'utf8'));

console.log('📋 Dependencias actuales:');
console.log(JSON.stringify(packageJson.dependencies, null, 2));

// Verificar dependencias específicas del proyecto
const requiredDeps = ['express', 'glob', 'handlebars', 'shiki', 'gray-matter', 'express-handlebars'];
const missingDeps = requiredDeps.filter(dep => !packageJson.dependencies[dep]);

if (missingDeps.length > 0) {
    console.log('❌ Dependencias faltantes:', missingDeps);
} else {
    console.log('✅ Todas las dependencias requeridas están presentes');
}

## 4. Ejecutar npm install

Instalamos todas las dependencias del proyecto desde cero.

In [ ]:
# Limpiar instalación anterior
echo "🧹 Limpiando node_modules anterior..."
rm -rf node_modules package-lock.json

echo "📦 Instalando todas las dependencias..."
npm install

echo "\n🔍 Verificar instalación completa:"
npm list --depth=0

## 5. Verificar Versiones de Node.js

Comprobamos la compatibilidad entre Node.js y las dependencias.

In [ ]:
# Verificar versiones
echo "🟢 Versión de Node.js:"
node --version

echo "\n📦 Versión de npm:"
npm --version

echo "\n⚙️ Engines especificados en package.json:"
cat package.json | grep -A 5 '"engines"'

echo "\n🔧 Verificar compatibilidad con glob:"
npm info glob engines

## 6. Solucionar Problemas de Rutas de Módulos

Diagnosticamos problemas relacionados con rutas de módulos y estructura de directorios.

In [ ]:
// Verificar rutas de módulos
console.log('📂 Directorio actual:', process.cwd());
console.log('🔍 Rutas de módulos de Node.js:');
console.log(module.paths);

// Intentar require del módulo glob
try {
    const glob = require('glob');
    console.log('✅ Módulo glob cargado correctamente');
    console.log('📍 Ubicación del módulo:', require.resolve('glob'));
} catch (error) {
    console.log('❌ Error al cargar glob:', error.message);
}

// Verificar estructura de directorios
const fs = require('fs');
console.log('\n📁 Estructura del proyecto:');
const files = fs.readdirSync('.', { withFileTypes: true });
files.forEach(file => {
    const icon = file.isDirectory() ? '📁' : '📄';
    console.log(`${icon} ${file.name}`);
});

## 🚀 Solución Aplicada

### Problema Principal
Render estaba ejecutando el build sin instalar las dependencias primero.

### Soluciones Implementadas
1. **Modificar el comando de build** en Render para incluir instalación de dependencias
2. **Actualizar package.json** con engines específicos
3. **Crear script de build personalizado** para Render

### Configuración Final en Render
- **Build Command:** `npm ci && npm run build`
- **Start Command:** `npm start`
- **Node Version:** 18+ (especificado en engines)

## 🎭 Nuevo Error: Playwright no encontrado

### Problema Detectado
```
> playwright install chromium
sh: 1: playwright: not found
npm error code 127
```

El script `postinstall` se ejecuta inmediatamente después de `npm ci`, pero Playwright aún no está disponible en el PATH.

In [ ]:
// Solución: Modificar el postinstall para usar npx
// El problema es que playwright no está en el PATH cuando se ejecuta postinstall

console.log('🔧 Diagnosticando problema de Playwright...');

// Verificar si playwright está instalado
const fs = require('fs');
const path = require('path');

const playwrightPath = path.join('node_modules', '.bin', 'playwright');
const packageJsonPath = 'package.json';

if (fs.existsSync(packageJsonPath)) {
    const packageJson = JSON.parse(fs.readFileSync(packageJsonPath, 'utf8'));
    console.log('📦 Scripts actuales:', packageJson.scripts);
    
    // Mostrar la solución
    const newScripts = {
        ...packageJson.scripts,
        "postinstall": "npx playwright install chromium"
    };
    
    console.log('✅ Script corregido:', newScripts.postinstall);
}

## ✅ Solución Final Implementada

### 🔧 Problemas Resueltos:

1. **Error "Cannot find module 'glob'"**
   - ✅ Solucionado: Usar `npm ci && npm run build` en Render

2. **Error "playwright: not found"**  
   - ✅ Solucionado: Usar `npx playwright install chromium`
   - ✅ Añadido manejo de errores con `|| echo 'continuing...'`

### 🎯 Configuración Final en Render:

**Build Command:** `npm ci && npm run build`
**Start Command:** `npm start`

### 📁 Archivos Creados:

1. `src/generate-safe.js` - Versión con fallback sin Playwright
2. `build.sh` - Script robusto de build con manejo de errores
3. `render-troubleshooting.ipynb` - Este notebook de diagnóstico

### 🚀 Scripts Disponibles:

- `npm run build` - Build principal (usa generate-safe.js)
- `npm run build:simple` - Solo HTML sin PDF
- `npm run build:with-script` - Usa build.sh completo

### 🎭 Manejo de Playwright:

- Si Playwright falla, el build continúa generando solo HTML
- El PDF se genera solo si Playwright está disponible
- Error handling robusto para evitar fallos completos

## 🎯 Progreso Actual - Playwright Instalado Correctamente

### ✅ Lo que YA funciona:
- ✅ Dependencias instaladas (`npm ci` exitoso)
- ✅ Playwright descargado (Chromium 140.0.7339.16)
- ✅ 179 packages auditados sin vulnerabilidades

### ❌ Problema Actual:
```
Error: Cannot find module 'playwright'
Require stack: /opt/render/project/src/src/generate.js
```

**Causa:** Render está ejecutando `build:simple` que usa `generate.js` (original) en lugar de `generate-safe.js` (con fallback)

In [ ]:
// 🔧 SOLUCIÓN DEFINITIVA: Mover Playwright a dependencies

const fs = require('fs');
const packageJson = JSON.parse(fs.readFileSync('package.json', 'utf8'));

console.log('🎯 Problema identificado:');
console.log('Playwright estaba en devDependencies, pero Render solo instala dependencies en producción');

console.log('\n✅ Solución aplicada:');
console.log('1. Movido playwright de devDependencies a dependencies');
console.log('2. Cambiado build:simple para usar generate-safe.js');
console.log('3. Creado RENDER-DEPLOY.md con instrucciones');

console.log('\n📦 Dependencies actuales:');
Object.keys(packageJson.dependencies).forEach(dep => {
    console.log(`  ✓ ${dep}: ${packageJson.dependencies[dep]}`);
});

console.log('\n🚀 Build command correcto para Render:');
console.log('npm ci && npm run build');